In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
solver_loc = 'C:/Users/ju116e/Desktop/Pyomo/glpk-4.65/w64/glpsol'

# Getting the Distance Matrix

In [2]:
file = open('17.txt')
lines = file.readlines()
file.close()

distance_matrix = []
for i in range(len(lines)):
    aux = lines[i][:-1].split('\t')
    aux = [int(i) for i in aux if i!= '']
    distance_matrix.append(aux)

n = 17

In [3]:
import numpy as np
distance = np.array(distance_matrix)

# MODEL INITIALIZATION AND ITS PARAMETERS

In [4]:
#Model
model = ConcreteModel()

In [5]:
M = np.arange(1, n, 1)
N = np.arange(1, n, 1)
U = np.arange(2, n, 1)

In [6]:
# #Indexes for the cities
# model.M = RangeSet(n)                
# model.N = RangeSet(n)

# #Index for the dummy variable u
# model.U = RangeSet(2,n)

In [7]:
#Decision variables xij
model.x=Var(N,M, within=Binary)

#Dummy variable ui
model.u=Var(N, within=NonNegativeIntegers,bounds=(0,n-1))

In [8]:
#Distance Matrix cij
model.c = Param(N, M,initialize=lambda model, i, j: distance[i-1][j-1])

# OBJECTIVE FUNCTION AND CONSTRAINTS

In [9]:
def obj_func(model):
    return sum(model.x[i,j] * distance[i-1][j-1] for i in N for j in M)

model.objective = Objective(rule=obj_func,sense=minimize)

In [10]:
##------------------------------------------------------##
#Only 1 leaves each city

def rule_const1(model,j):
    return sum(model.x[i,j] for i in N if i!=j ) == 1

model.const1 = Constraint(M,rule=rule_const1)

In [11]:
##------------------------------------------------------##
#Only 1 enters each city

def rule_const2(model,i):
    return sum(model.x[i,j] for j in M if j!=i) == 1

model.rest2 = Constraint(N,rule=rule_const2)

In [12]:
##------------------------------------------------------##
#Only a single tour covering all cities

def rule_const3(model,i,j):
    if i!=j: 
        return model.u[i] - model.u[j] + model.x[i,j] * n <= n-1
    else:
        #Yeah, this else doesn't say anything
        return model.u[i] - model.u[i] == 0 
    
model.rest3 = Constraint(U,N,rule=rule_const3)

# MODEL SOLVING

In [13]:
#Prints the entire model
model.pprint()

12 Set Declarations
    c_index : Dim=0, Dimen=2, Size=256, Domain=None, Ordered=False, Bounds=None
        Virtual
    c_index_0 : Dim=0, Dimen=1, Size=16, Domain=None, Ordered=False, Bounds=(1, 16)
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    c_index_1 : Dim=0, Dimen=1, Size=16, Domain=None, Ordered=False, Bounds=(1, 16)
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    const1_index : Dim=0, Dimen=1, Size=16, Domain=None, Ordered=False, Bounds=(1, 16)
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    rest2_index : Dim=0, Dimen=1, Size=16, Domain=None, Ordered=False, Bounds=(1, 16)
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    rest3_index : Dim=0, Dimen=2, Size=240, Domain=None, Ordered=False, Bounds=None
        Virtual
    rest3_index_0 : Dim=0, Dimen=1, Size=15, Domain=None, Ordered=False, Bounds=(2, 16)
        [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    rest3_index_1 : Dim=0, Dimen=1, Si

        Key : Lower : Body                                                                                                                                                      : Upper : Active
          1 :   1.0 :               x[1,2] + x[1,3] + x[1,4] + x[1,5] + x[1,6] + x[1,7] + x[1,8] + x[1,9] + x[1,10] + x[1,11] + x[1,12] + x[1,13] + x[1,14] + x[1,15] + x[1,16] :   1.0 :   True
          2 :   1.0 :               x[2,1] + x[2,3] + x[2,4] + x[2,5] + x[2,6] + x[2,7] + x[2,8] + x[2,9] + x[2,10] + x[2,11] + x[2,12] + x[2,13] + x[2,14] + x[2,15] + x[2,16] :   1.0 :   True
          3 :   1.0 :               x[3,1] + x[3,2] + x[3,4] + x[3,5] + x[3,6] + x[3,7] + x[3,8] + x[3,9] + x[3,10] + x[3,11] + x[3,12] + x[3,13] + x[3,14] + x[3,15] + x[3,16] :   1.0 :   True
          4 :   1.0 :               x[4,1] + x[4,2] + x[4,3] + x[4,5] + x[4,6] + x[4,7] + x[4,8] + x[4,9] + x[4,10] + x[4,11] + x[4,12] + x[4,13] + x[4,14] + x[4,15] + x[4,16] :   1.0 :   True
          5 :   1.0 :              

In [ ]:
import time
from pytz import timezone
import datetime as dt

coffee_time = time.time()
print('Start Time is',dt.datetime.now(timezone('Asia/Calcutta')).strftime("%H:%M:%S"))

#------------------------ Solving Using Pyomo --------------------------------------------
#Solves
result = SolverFactory('glpk',executable = solver_loc).solve(model)
#-----------------------------------------------------------------------------------------


print("Time required for",n,"nodes TSP is %s minutes ---" % ((time.time() - coffee_time) / 60))

Start Time is 21:26:35


In [ ]:
model.objective()

In [ ]:
#Prints the results
# print(result)

# PRINT OF THE CHOSEN DECISION VARIABLES

In [ ]:
List = list(model.x.keys())
for i in List:
    if model.x[i]() != 0:
        print(i,'--', model.x[i]())